# Final CHO Model
This notebook is to asses the validity of our reconstruction and how complete it is.

In [1]:
# Import libraries
import gspread
import pandas as pd
import numpy as np
from cobra import Model, Reaction, Metabolite
from cobra.io import validate_sbml_model, write_sbml_model

/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 1. Retrieve information from the Google Sheet datasets reactions and metabolites

In [2]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns and Metabolites sheet.
rxns_sheet = cho_recon.worksheet('Rxns')
rxns_attributes_sheet = cho_recon.worksheet('Attributes')
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create pd DataFrames

# Reactions IDs, names, formulas, GPRs
rxns = pd.DataFrame(rxns_sheet.get_all_records())

# Reactions bounds
rxns_attributes = pd.DataFrame(rxns_attributes_sheet.get_all_records())

# Metabolites names, formulas and compartment
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())

### 2. Build a model and feed it the information from the df generated

In [3]:
##### ----- Create a model and add reactions ----- #####
model = Model("iCHO")
lr = []

for _, row in rxns.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Name,iCHO
Memory address,151893910
Number of metabolites,0
Number of reactions,8189
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [4]:
##### ----- Add information to each one of the reactions ----- #####
for i,r in enumerate(model.reactions):
    r.build_reaction_from_string(rxns['Reaction Formula'][i])
    r.name = rxns['Reaction Name'][i]
    r.subsystem = rxns['Subsystem'][i]
    r.gene_reaction_rule = str(rxns['GPR_final'][i])
    r.lower_bound = float(rxns_attributes['Lower bound'][i])
    r.upper_bound = float(rxns_attributes['Upper bound'][i])
    # print(r)

unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
unknown metabolite '10fthf5glu_m' created
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
unknown metabolite '10fthf6glu_m' created
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
unknown metabolite '10fthf7glu_m' created
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
unknown metabolite '10fthf_m' created
unknown metabolite '11docrtsl_c' created
unknown metabolite '11docrtsl_m' created
unknown metabolite '11docrtsl_r' created
unknown metabolite '11docrtstrn_c' created
unknown metabolite '11docrtstrn_m' created
unknown metabolite '11docrtstrn_r' created
unknown metabolite '12HPET_c' created
unknown metabolite 'atp_c' created
unknown metabolite 'h2o_c' created
unknown metabolite '12HPET_e' created
unknown metabolite 'adp_c' created
unknown metabolite 'h_c' created
unknown metabolite 'pi_c' created
unknown 

unknown metabolite '3mob_e' created
unknown metabolite '3mop_c' created
unknown metabolite '3mop_e' created
unknown metabolite '3mox4hoxm_c' created
unknown metabolite '3mox4hoxm_e' created
unknown metabolite '3m4hpga_c' created
unknown metabolite '3moxtyr_c' created
unknown metabolite '3amp_l' created
unknown metabolite 'h2o_l' created
unknown metabolite 'adn_l' created
unknown metabolite 'pi_l' created
unknown metabolite '3odecACP_c' created
unknown metabolite '3odecACP_m' created
unknown metabolite '3oddecACP_c' created
unknown metabolite '3oddecACP_m' created
unknown metabolite '3omrsACP_c' created
unknown metabolite '3omrsACP_m' created
unknown metabolite '3opalmACP_c' created
unknown metabolite '3ooctdACP_c' created
unknown metabolite 'actACP_c' created
unknown metabolite 'actACP_m' created
unknown metabolite '3ohexACP_c' created
unknown metabolite '3ohexACP_m' created
unknown metabolite '3ooctACP_c' created
unknown metabolite '3ooctACP_m' created
unknown metabolite 'malACP_c' cr

unknown metabolite 'c142_5Z_8Zcoa_x' created
unknown metabolite 'c143_5Z_8Z_11Zcoa_m' created
unknown metabolite 'c143_5Z_8Z_11Zcoa_x' created
unknown metabolite 'ptdcacoa_m' created
unknown metabolite 'pmtcoa_m' created
unknown metabolite 'pmtcoa_x' created
unknown metabolite 'hdcoa_m' created
unknown metabolite 'hxdccoa_m' created
unknown metabolite 'hxdccoa_x' created
unknown metabolite 'thexddcoa_m' created
unknown metabolite 'c162_7Z_10Zcoa_m' created
unknown metabolite 'c162_7Z_10Zcoa_x' created
unknown metabolite 'c163_7Z_10Z_13Zcoa_m' created
unknown metabolite 'c163_7Z_10Z_13Zcoa_x' created
unknown metabolite 'c163_4Z_7Z_10Zcoa_m' created
unknown metabolite 'c163_4Z_7Z_10Zcoa_x' created
unknown metabolite 'c164_4Z_7Z_10Z_13Zcoa_m' created
unknown metabolite 'c164_4Z_7Z_10Z_13Zcoa_x' created
unknown metabolite 'accoa_c' created
unknown metabolite 'aacoa_m' created
unknown metabolite 'aacoa_x' created
unknown metabolite 'stcoa_m' created
unknown metabolite 'stcoa_x' created
unkn

unknown metabolite 'Sdmhptcoa_m' created
unknown metabolite 'Sdmnoncoa_m' created
unknown metabolite 'mptcoa_m' created
unknown metabolite 'Spristcoa_x' created
unknown metabolite 'Stmhndccoa_x' created
unknown metabolite 'Stmtridccoa_x' created
unknown metabolite 'dcacoa_c' created
unknown metabolite 'dca_c' created
unknown metabolite 'dca_m' created
unknown metabolite 'dca_x' created
unknown metabolite 'ddcacoa_c' created
unknown metabolite 'ddca_c' created
unknown metabolite 'ddca_m' created
unknown metabolite 'ddca_x' created
unknown metabolite 'tdcoa_c' created
unknown metabolite 'ttdca_c' created
unknown metabolite 'ttdca_m' created
unknown metabolite 'ttdca_x' created
unknown metabolite 'pmtcoa_c' created
unknown metabolite 'hdca_c' created
unknown metabolite 'hdca_m' created
unknown metabolite 'hdcoa_c' created
unknown metabolite 'hdcea_c' created
unknown metabolite 'hdcea_m' created
unknown metabolite 'hdcoa_x' created
unknown metabolite 'hdcea_x' created
unknown metabolite 'o

unknown metabolite 'id3acald_c' created
unknown metabolite 'ind3ac_c' created
unknown metabolite 'id3acald_m' created
unknown metabolite 'ind3ac_m' created
unknown metabolite 'pristanal_c' created
unknown metabolite 'prist_c' created
unknown metabolite 'aldstrn_c' created
unknown metabolite 'aldstrn_e' created
unknown metabolite 'aldstrn_m' created
unknown metabolite 'dhap_c' created
unknown metabolite 'dha_c' created
unknown metabolite '12RHPET_c' created
unknown metabolite '5HPET_c' created
unknown metabolite 'leuktrA4_c' created
unknown metabolite 'mag_cho_m' created
unknown metabolite 'am6sa_c' created
unknown metabolite 'amuco_c' created
unknown metabolite 'dhcholestancoa_x' created
unknown metabolite 'dmnoncoa_m' created
unknown metabolite 'pristcoa_x' created
unknown metabolite 'tmtridccoa_x' created
unknown metabolite 'cholcoar_x' created
unknown metabolite 'ethamp_c' created
unknown metabolite 'mnh4p_c' created
unknown metabolite 'amet_r' created
unknown metabolite 'amet_m' cr

...

unknown metabolite 'CE6458_c' created
unknown metabolite 'CE6459_c' created
unknown metabolite 'CE6448_c' created
unknown metabolite 'CE6460_c' created
unknown metabolite 'CE6461_c' created
unknown metabolite 'CE6462_c' created
unknown metabolite 'CE6463_c' created
unknown metabolite 'CE6464_c' created
unknown metabolite 'CE6465_c' created
unknown metabolite 'CE6467_c' created
unknown metabolite 'CE6466_c' created
unknown metabolite 'CE0074_c' created
unknown metabolite 'urate_n' created
unknown metabolite 'CE0074_n' created
unknown metabolite 'urea_n' created
unknown metabolite 'C02147_c' created
unknown metabolite 'CE2592_c' created
unknown metabolite 'CE0853_c' created
unknown metabolite 'CE1102_c' created
unknown metabolite 'CE0782_c' created
unknown metabolite 'CE2597_c' created
unknown metabolite 'CE0692_c' created
unknown metabolite 'CE0955_r' created
unknown metabolite 'pail45p_cho_g' created
unknown metabolite 'pail45p_cho_r' created
unknown metabolite 'pail35p_cho_n' created


unknown metabolite 'CE5319_c' created
unknown metabolite 'CE5319_m' created
unknown metabolite 'CE5319_x' created
unknown metabolite 'CE5342_c' created
unknown metabolite 'CE5342_r' created
unknown metabolite 'CE7144_c' created
unknown metabolite 'CE7145_c' created
unknown metabolite 'CE7144_r' created
unknown metabolite 'CE7145_r' created
unknown metabolite 'CE5956_c' created
unknown metabolite 'CE5853_l' created
unknown metabolite 'CE1925_l' created
unknown metabolite 'CE6415_c' created
unknown metabolite 'C01601_c' created
unknown metabolite 'CE2006_c' created
unknown metabolite 'CE6506_c' created
unknown metabolite 'CE6504_c' created
unknown metabolite 'C08261_c' created
unknown metabolite 'C05957_m' created
unknown metabolite 'CE4876_m' created
unknown metabolite 'CE4876_r' created
unknown metabolite 'CE5304_c' created
unknown metabolite 'CE4876_c' created
unknown metabolite 'CE5944_c' created
unknown metabolite 'CE5944_m' created
unknown metabolite 'CE5944_x' created
unknown meta

unknown metabolite 'retncoa_c' created
unknown metabolite 'anhretinol_c' created
unknown metabolite 'retnglc_c' created
unknown metabolite '13_cis_retnglc_c' created
unknown metabolite '13_cis_retnglc_r' created
unknown metabolite 'retnglc_r' created
unknown metabolite 'selmeth_c' created
unknown metabolite 'C05708_c' created
unknown metabolite 'selmeth_r' created
unknown metabolite 'C05708_r' created
unknown metabolite '5mthf_n' created
unknown metabolite 'selhcys_n' created
unknown metabolite 'selmeth_n' created
unknown metabolite 'thf_n' created
unknown metabolite '5mthf_r' created
unknown metabolite 'selhcys_r' created
unknown metabolite 'thf_r' created
unknown metabolite 'nrpphr_r' created
unknown metabolite 'CE5538_r' created
unknown metabolite 'C07535_c' created
unknown metabolite 'CN0012_c' created
unknown metabolite 'C07535_r' created
unknown metabolite 'CN0012_r' created
unknown metabolite 'CN0009_c' created
unknown metabolite 'CN0009_r' created
unknown metabolite 'CN0012_x' 

unknown metabolite 'thmtp_c' created
unknown metabolite 'thm_m' created
unknown metabolite 'thrtrna_m' created
unknown metabolite 'trnathr_m' created
unknown metabolite 'thr_L_l' created
unknown metabolite 'thyoxs_c' created
unknown metabolite 'triodthysuf_c' created
unknown metabolite 'tmlys_c' created
unknown metabolite 'tmndnccrn_c' created
unknown metabolite 'tmndnccrn_m' created
unknown metabolite 'xtra_e' created
unknown metabolite 'trdox_m' created
unknown metabolite 'trdrd_m' created
unknown metabolite 'indpyr_c' created
unknown metabolite 'trptrna_m' created
unknown metabolite 'trnatrp_m' created
unknown metabolite 'trp_L_m' created
unknown metabolite 'trp_L_l' created
unknown metabolite 'ts3_c' created
unknown metabolite 'tststeroneglc_c' created
unknown metabolite 'tststeroneglc_r' created
unknown metabolite 'tststerones_c' created
unknown metabolite 'ttdcrn_m' created
unknown metabolite 'txa2_c' created
unknown metabolite 'txb2_c' created
unknown metabolite '11dhtxb2_c' cre

unknown metabolite 'fdxrd_c' created
unknown metabolite 'fdxo_2_2_c' created


In [5]:
##### ----- Add information for each metabolite ----- #####
metabolites_dict = metabolites.set_index('BiGG ID').to_dict('dict')
metabolites_dict['Name']

for met in model.metabolites:
    met.name = metabolites_dict['Name'][f'{met}']
    met.formula = metabolites_dict['Formula'][f'{met}']
    met.compartment = metabolites_dict['Compartment'][f'{met}'].split(' - ')[0]

In [6]:
##### ----- Build the S matrix ----- #####
num_metabolites = len(model.metabolites)
num_reactions = len(model.reactions)
S = np.zeros((num_metabolites, num_reactions))
for i, metabolite in enumerate(model.metabolites):
    for j, reaction in enumerate(model.reactions):
        S[i, j] = reaction.metabolites.get(metabolite.id, 0)


In [54]:
##### ----- Save the model ----- #####
write_sbml_model(model, "iCHOv2.xml")

### Blocked reactions and Dead-Ends

In [8]:
##### ----- FVA ----- #####
import importlib
# from utils import runMinMax_GF
import utils
importlib.reload(utils)

for rxn in model.reactions:
    rxn.bounds = -1000, 1000
minmax = utils.runMinMax_Single(model, end_rxn_index=None)

500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000


In [11]:
import importlib
import utils
importlib.reload(utils)
dead_ends = utils.detect_dead_ends(S)

# Detect dead-end metabolites
is_dead_end = utils.detect_dead_ends(S)

# Get the indices of the dead-end metabolites
dead_end_indices = np.nonzero(is_dead_end)[0]

# Get the names of the dead-end metabolites
dead_end_metabolites = [model.metabolites[i].id for i in dead_end_indices]

# Print the names of the dead-end metabolites
with open('Dead_ends.txt', 'w') as f:
    for dead_met in dead_end_metabolites:
        print(dead_met, file=f)

In [13]:
##### Print Reactions with the min and max fluxes #####
with open('FVA_Results.txt', 'w') as f:
    for i, j in enumerate(minmax):
        print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)
        
##### Print Blocked Reactions #####
with open('Blocked_Reactions.txt', 'w') as f:
    for i, j in enumerate(minmax):
        if j[0] == 0 and j[1] == 0:
            print(model.reactions[i].id, "Min: ",j[0], "Max: ", j[1], file=f)



In [14]:
# Check Mass Balance
for rxn in model.reactions:
    rxn.check_mass_balance()

In [13]:
# Initiliase model
for rxn_exchange in model.exchanges:
    rxn_exchange.bounds = (-1000, 1000)
    
bio_id = model.reactions.index('biomass_producing')
atp_id = model.reactions.index('DM_atp_c')

model.reactions[bio_id].bounds = (0, 100)
model.objective = 'biomass_producing'
model.objective = 'DM_atp_c'
model.optimize()

(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000, 1000)
(-1000

In [149]:
with open('Biomass_Metabolites_Reactions.txt', 'w') as f:
    bio_mets = model.reactions.get_by_id('biomass_producing').metabolites
    for bio_met in bio_mets:
        f.write("Metabolite: " + bio_met.name + "\n")
        f.write("Reactions: " + ", ".join([r.name for r in bio_met.reactions]) + "\n")
        f.write("Formula: " + ", ".join([r.build_reaction_string() for r in bio_met.reactions]) + "\n\n")

# Detect all the excahnge reactions that have no uptake
from cobra.flux_analysis import flux_variability_analysis
FVA_Result = flux_variability_analysis(model, model.exchanges)

FVA_Result = a[a.minimum == 0]
with open('FVA_Excange.txt', 'w') as f:
    f.write("reaction\tminimum\tmaximum\n")
    for index, row in FVA_Result.iterrows():
        f.write(f"{index}\t{row['minimum']}\t{row['maximum']}\n")

In [ ]:
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

10FTHF5GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF5GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtr blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtr blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETATP blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETUPKt blocked (bounds: (0, 0)), new growth rate 0.000000
13DAMPPOX blocked (bounds: (0, 0)), new growth rate 0.000000
15HPETATP blocked (bounds: (0, 0)), new growth rate 0.00

In [ ]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)